## EANBiT Residential Training 2021


### Link to the data
[This](https://www.dropbox.com/s/nkyunpxtwa4s0td/recov_data.zip?dl=0) is the link to the fast5 files we will use for this training.


[This](https://drive.google.com/file/d/1Jyf_yMIhor7NnqhbIfBJSBlRTuKvJvhx/view?usp=sharing) is the link to the basecalled fastq files.


In [1]:
#Install bash kernel
import sys
!{sys.executable} -m pip install bash-kernel

In [2]:
#jupyter console --kernel bash
!jupyter kernelspec list

Available kernels:
  python3    C:\Users\SecondFiddle\Anaconda3\share\jupyter\kernels\python3


### Visualizing fast5 files using Bulkvis
BulkVis is a program written in Python3 using Bokeh to visualise raw squiggle data from Oxford Nanopore Technologies (ONT) bulkfiles. [Bulkvis](https://github.com/LooseLab/bulkvis) scans a folder containing bulk FAST5 files at startup. An individual file is selected and specific channels plotted. Basic metadata are displayed to the user. To navigate coordinates are input in the format `channel: start_time-end_time`. Alternatively pasting a FASTQ read header from a base called read will jump to its channel, time and raw signal from the bulk FAST5 file (Payne et al., 2019). Further information on how to strt bulkvis can be found [here](https://bulkvis.readthedocs.io/en/latest/quickstart.html).

Before you install bulkvis, make sure you have pip installed.
To check whether you have pip installed, run this command:

`pip --version`

If pip is not installed, no worries! Let's fix that.
Let's get on with the installation.
To install pip:

In [ ]:
## On Debian/Ubuntu
apt install python3-pip
## On CentOS
yum install epel-release 
yum install python-pip
## Using conda 
conda install -c anaconda pip

Once you have `pip` installed, you will be required to create and activate a virtual environment to work on.
To do this, follow [these](https://bulkvis.readthedocs.io/en/latest/installation.html) simple instructions. Please edit the paths to match yours where need be, and make sure you are using the **bulk fast5 file** not a single fast5 file. This bulk fast 5 file is obtained by setting up the sequencing run as shown [here](https://github.com/LooseLab/bulkvis/issues/28) on MinKNOW.

### Basecalling
fast5 files generated from the sequencing step are basecalled
##### Basecalling script
This script was developed by [Sirselim](https://gist.github.com/sirselim) for faster basecalling with the free GPU resources on [Google collaboratory](https://research.google.com/colaboratory/). If you would like to try out/use this resource, please use your google account to open google collaboratory, and  create a notebook of your own to implement the script below.

[This](https://gist.github.com/sirselim/13f70ae69f2a512e7d9e1f00f9704f53) is the link to his basecalling script on his github page. However, there is an alternative to use CPU resources, whose basecalling capabilities are way slower.

Alternatively, we can perform CPU basecalling as shown below. It is recommended you run this step on the HPC where more threads can be allocated per caller. 

Remotely sign into your HPC using your ssh address: 

In [ ]:
ssh <username>@host.address

Guppy can be installed using the code shown below:

In [ ]:
%%shell
#Install Guppy
GuppyBinary=("https://mirror.oxfordnanoportal.com/software/analysis/ont-guppy_5.0.11_linux64.tar.gz")
wget $GuppyBinary

Unpack the guppy binary files using the following command:

In [ ]:
%%shell
tar -xvzf ont-guppy_5.0.11_linux64.tar.gz

To perform the basecalling step you need to know the [flowcell and ONT kit](https://denbi-nanopore-training-course.readthedocs.io/en/latest/basecalling/basecalling.html) used to generate your fast5 files. In this case the kit used was SQK-PCB109, and FLO-MIN106 flowcell. The config file used for this combination is:
**dna_r9.4.1_450bps_hac**

In [ ]:
guppy_basecaller --compress_fastq -i ~/workdir/data/fast5_tiny/ \
-s ~/workdir/basecall_tiny/ --cpu_threads_per_caller 14 --num_callers 1 -c dna_r9.4.1_450bps_hac.cfg

If you encounter a `libzmq.so.5` error go [here](https://github.com/bulwark-crypto/Bulwark/issues/28) and use the code chunk to recompile zeromq library 

We will be using the script [ONT_analysis.sh](https://github.com/eKariuki-sleepy/ONT_Analysis/blob/main/ONT_analysis.sh) for basecalling. Please edit the variables accordingly to suit your directory structure. The exercise will take approximately 25 minutes running on 16 threads and one-caller. [This](https://denbi-nanopore-training-course.readthedocs.io/en/latest/basecalling/basecalling.html) resource provides more information about basecalling with guppy.

### Obtain Quality statistics with pycoQC
[PycoQC](https://github.com/tleonardi/pycoQC) computes metrics and generates interactive QC plots for Oxford Nanopore technologies sequencing data.

PycoQC relies on the sequencing_summary.txt file generated by Albacore and Guppy (hence limited input options), but if needed it can also generate a summary file from basecalled fast5 files. The package supports 1D and 1D2 runs generated with Minion, Gridion and Promethion devices, basecalled with Albacore 1.2.1+ or Guppy 2.1.3+ (Leger & Leonardi, 2019). 

In [ ]:
%%shell
echo 'Running pycoQC ...'

pycoQC -f $SEQ_SUMMARY -o $RES/pycoqc.html

In [ ]:
#'Convert basecalled fastqs.gz into one .qz file ...'

gunzip -c $FASTQs/*.gz | gzip > $RES/reads.fastq.gz

### Filter Low-quality reads with nanofilt

[Nanofilt](https://github.com/wdecoster/nanofilt) is a tool written for Python 3 but executable in bash. It performs filtering on quality and/or read length, and optional trimming after passing filters (De Coster et al., 2018).
Intended to be used:

 - directly after fastq extraction
 - prior to mapping
 - in a stream between extraction and mapping

In [ ]:
#Install nanofilt
#conda
conda install -c bioconda nanofilt
#pip
pip install nanofilt

In [ ]:
#This below runs Nanofilt, trimming quality at <int>7> (based on pycoQc analysis).
#Remember a quality of 10 indicates 90% accuracy.
gunzip -c reads.fastq.gz | NanoFilt -q 10 | gzip > highQuality-reads.fastq.gz

### Adapter and Quality Trimming with Porechop

[Porechop](https://github.com/rrwick/Porechop) is a tool for finding and removing adapters from Oxford Nanopore reads. Adapters on the ends of reads are trimmed off, and when a read has an adapter in its middle, it is treated as chimeric and chopped into separate reads. Porechop performs thorough alignments to effectively find adapters, even at low sequence identity (Wick et al., 2017).

Begin by cloning the repo (below). Running the setup.py script will compile the C++ components of Porechop and install a porechop executable:

In [ ]:
#Clone the repo
git clone https://github.com/rrwick/Porechop.git
# move to the porechop working directory
cd Porechop
#Run the script setup.py to compile and install porechop
python3 setup.py install
#Check whether porechop has been successfully installed, as well as how its used
porechop -h

Basic adapter trimming can be done as follows:

In [ ]:
porechop -i $fastq_dir -o output_reads.fastq.gz

### NanoPlot

[Nanoplot](https://github.com/wdecoster/NanoPlot) is also a plotting tool for long read sequencing data and alignments (De Coster et al., 2018). It takes more input formats than pycoQC including multiple `.fastq files`. Once your reads have been trimmed and filtered NanoPlot can be run using the following command:

In [ ]:
%%shell
NanoPlot -t 20 --fastq $RES/adfree_reads.fastq.gz --N50 -o $RES \
--maxlength 40000 --plots dot --legacy hex

### Getting the Reference Genome for mapping.
Our data is sourced from the Black Soldier Fly (BSF). The BSF reference genome can be found [here](https://www.ncbi.nlm.nih.gov/assembly/GCF_905115235.1/)

## _De novo_ Genome Assembly

Advances in DNA sequencing has enabled the rapid analysis of genomes, driving biological discovery. However, sequencing of complex genomes, which are very large and have a high content of repetitive sequences or many copies of similar sequences, still remains a great challenge. With improvements in long-read sequencing however, it is easier to generate high-quality sequences for complex genomes. The **Overlap Layout Consensus algorithm (OLC)** is best suited for long-read sequencing technologies like [PacBio](https://www.pacb.com/smrt-science/smrt-sequencing/) and [ONT](https://nanoporetech.com/applications/dna-nanopore-sequencing).

TGS generate read lengths of over 10,000 base-pairs that can be advantageously used to improve the genome assembly.Long reads can span stretches of repetitive regions and thus produce a more
contiguous reconstruction of the genome.

There are two main families of assemblers based on long reads:
1. Long Reads Only assembler (LRO)
1. Short and Long Reads combined assembler (SLR)

LRO assemblers are based on the OLC algorithm. SLR assemblers instead generate a de Bruijn graph pre-assembly using short reads, then the long reads are used to improve the pre-assembly by closing gaps, ordering contigs, and resolving repetitive regions.


## OLC

![](https://www.researchgate.net/profile/Christina-Toft/publication/26266221/figure/fig2/AS:216492458156039@1428627232413/Overlap-layout-consensus-genome-assembly-algorithm-Reads-are-provided-to-the-algorithm.png)


Resource: [Long-read tools](https://long-read-tools.org/index.html)


## Requirements

* [minimap2](https://github.com/lh3/minimap2): All-vs-all overlap
* [Miniasm](https://github.com/lh3/miniasm): Layout 
* [Minipolish](https://github.com/rrwick/Minipolish): Consenus

* [QUAST](http://quast.sourceforge.net/docs/manual.html): Quality Assesment



### Dataset

We will use the [African Swine Fever Virus (ASFV)](https://mra.asm.org/content/9/44/e00948-20) genome which is double stranded with a size of 170 to 194 kbp. Data used was from a Virulent African Swine Fever Virus from a Domestic Pig in Ukraine under the SRA accession number [SRX6477592](https://www.ncbi.nlm.nih.gov/sra/SRX6477592[accn]). The preprocess step involved mapping the reads to the Sus scrofa 11 reference genome [GCA_000003025](https://www.ncbi.nlm.nih.gov/assembly/?term=GCF_000003025) to remove reads likely originating from the host. [minimap2](https://github.com/lh3/minimap2) map long noisy genomic reads will be used at this step. The SAM output was converted to BAM using [Samtools](http://www.htslib.org/). Umapped reads will be  filtered out using samtools and converted to fastq format. [Porechop](https://github.com/rrwick/Porechop) was used to trim adaptors discard and sequences with middle adapters. The chopped unmapped fastq reads will be used as the input in the pipeline.

Script for the preprocess step:


``` sh
#!/usr/bin/env bash
#SBATCH -p batch
#SBATCH -J Mini_PK
#SBATCH -n 16

#load the minimap2 module


module load minimap2/2.13


#run the minimap2 with 16 CPU threads (cores)

#mapping the Ukranian reads on the ref genome


minimap2 -ax map-ont /home/pkimani/Data/Sus_scrofa/GCF_000003025.6_Sscrofa11.1_genomic.fna /home/pkimani/Data/Ukraine/SRR9719895.fastq.gz > /home/pkimani/Data/Ukraine/SRR9719895.sam


#converting sam to bam

module load samtools/1.11

samtools view -O BAM /home/pkimani/Data/Ukraine/SRR9719895.sam -o /home/pkimani/Data/Ukraine/SRR9719895.bam


#filter unmapped reads

samtools view -b -f 4  /home/pkimani/Data/Ukraine/SRR9719895.bam >  /home/pkimani/Data/Ukraine/SRR9719895.unmapped.bam


#extract the fastq sequence

samtools fastq -0  /home/pkimani/Data/Ukraine/SRR9719895.unmapped.fastq /home/pkimani/Data/Ukraine/SRR9719895.unmapped.


#zip
gzip  /home/pkimani/Data/Ukraine/SRR9719895.unmapped.fastq 

# trim adapters
module load porechop/0.2.4

porechop -i /home/pkimani/Data/Ukraine/SRR9719895.unmapped.fastq.gz -o /home/pkimani/Data/Ukraine/SRR9719895.unmapped.chopped.fastq  --discard_middle

```

#### Processed dataset

Download the processed dataset [here](https://drive.google.com/file/d/1g4g1RNo40LWZx6oRebgVMjGdwR4qSd3K/view?usp=sharing) 

#### Reference genome

We will use the [reference genome](https://www.ncbi.nlm.nih.gov/nuccore/MN194591) from a Ukranian study with a length 191,911 bp to assess the quality of our assembly.

#### Overlap 

```sh
minimap2 -x ava-ont SRR9719895.unmapped.choped.fastq.gz SRR9719895.unmapped.choped.fastq.gz > SRR9719895.paf
```

#### Layout

```sh
miniasm -f  SRR9719895.unmapped.choped.fastq.gz SRR9719895.paf > SRR9719895.gfa
```

#### Concensus

```sh
minipolish  SRR9719895.unmapped.choped.fastq.gz SRR9719895.gfa  >SRR9719895_polished.gfa
```

#### convert gfa to fasta
```sh
awk '$1 ~/S/ {print ">"$2"\n"$3}' SRR9719895_polished.gfa > SRR9719895_polished.fasta
```


```sh
#!/bin/bash

#catch errors
set -eu

#read in the fastq file for mapping
read -p "Enter a path to the fastq file: " file

#check if the file exists or is empty

if [[ ! -e ${file} ]]
then 
	echo "file does not exist"

elif [[ ! -s ${file} ]]
then
	echo "file is empty"

else
	echo "file is in the right format"
	echo "Proceeding"

fi

output="output.paf"

echo " initiating assembly"

echo " Keep calm"

minimap2 -x ava-ont ${file} ${file} >  ${output}

echo " All-vs-All overlap complete"

echo " Initiating the miniasm"

output2="output.gfa"

miniasm -f ${file} ${output} > ${output2}

echo "Graph assembly complete"

echo "initiating the consensus step"

minipolish ${file} ${output2} > polished.gfa

#converts the gfa format to fasta format

awk '$1 ~/S/ {print ">"$2"\n"$3}' polished.gfa > polished.fasta

echo " Successfully assembled" 
```

#### Quality Assesment using QUAST

```sh
quast -o Quast -r {ref} SRR9719895_polished.fasta
```

[Results](file:///home/muchina/Genome_Assemblers/Assembly/Ukraine_repo/Quast_3/report.html)


## Variant calling and annotation of ONT alignment data


### Generate a .vcf file using Freebayes
**Freebayes** is a Bayesian genetic variant detector designed to find small polymorphisms, specifically SNPs (single-nucleotide polymorphisms), indels (insertions and deletions), MNPs (multi-nucleotide polymorphisms), and complex events (composite insertion and substitution events) smaller than the length of a short-read sequencing alignment (Garrison & Marth, 2012).

We are going to use [this](https://drive.google.com/file/d/1rVZKC4vD_a3ZxrjIu_BVaE8807_RZu5B/view?usp=sharing) bam file to generate a `.vcf` file.

#### Installing freebayes

In [ ]:
conda install -c bioconda freebayes

#### Running freebayes using .bam input

We are going to generate a .vcf file which we will later annotate using a .bam alignment file of the Black soldier Fly.

In [ ]:
freebayes -f genomic/ref.fa aln.bam >var.vcf

### Variant Annotation using snpEff
[**SnpEff**](http://pcingola.github.io/SnpEff/se_introduction/) is a variant annotation and effect prediction tool. It annotates and predicts the effects of genetic variants (such as amino acid changes).Genetic variants refer to differences between a genome and a "reference" genome. 


A typical snpEff operation use case is as follows:


**Input**: The inputs are predicted variants (SNPs, insertions, deletions and MNPs). The input file is usually obtained as a result of a sequencing experiment, and it is usually in variant call format (VCF).


**Output**: SnpEff analyzes the input variants. It annotates the variants and calculates the effects they produce on known genes (e.g. amino acid changes). 

The conda installation of snpEff is not very reliable. To install snpEff, run the following command then unzip the file:

`wget https://snpeff.blob.core.windows.net/versions/snpEff_latest_core.zip` 

The first step involves checking whether your genome exists in the snpEff database. Remember, snpEff is made using java, and to run it, you must either specify the full location of the `.jar` file, or run snpEff in the same directory as the `.jar` file.

In [ ]:
##Check whether snpEff is successfully installed.
java -jar snpEff.jar

## Check whether the snpEff database contains the genome whose variants you are annotating
## The .bam files are from Hermetia illucens, the Black soldier fly
java -jar snpEff.jar databases | grep Hermetia

### Building a snpEff database
If your database is not present, you will have to build your own database. There are many ways to build a snpEff database mainly dependent on the input data. In our case, we are going to use the `genomic.fna.gz` file and the `.gff` file.

To do this, do as follows:

Go [here](https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/905/115/235/GCF_905115235.1_iHerIll2.2.curated.20191125/GCF_905115235.1_iHerIll2.2.curated.20191125_genomic.gff.gz) and download the `.gff` file, and [here](https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/905/115/235/GCF_905115235.1_iHerIll2.2.curated.20191125/GCF_905115235.1_iHerIll2.2.curated.20191125_genomic.fna.gz) to download the `genomic.fna.gz` if you had not downloaded it.

Then follow the following steps:

**i. Edit the config file and add the prefix to your genome**. 
   
In order to tell SnpEff that there is a new genome available, you must update SnpEff's configuration file snpEff.config. The config file is present in the snpEff folder. Move to the end of the config file and add the genome prefix and its name. 
   In our case:


In [ ]:
###Edit the snpEff.config file and add the prefix to your genome. Add this as the last line of the file:
GCF_905115235.1.genome : Hermetia illucens, complete genome

### Create a directory for the new genome
cd /path/to/snpEff/data/
mkdir GCF_905115235.1

## Get the .gff file using wget, or if you have it downloaded mv it to the data folder.
mv /path/to/gff.gz .
## Rename the gff.gz to genes.gff.gz
mv gff.gz to genes.gff.gz


## Get the genome using wget, or if you have it downloaded, move it to the data folder into a new folder called genomes
mv /path/to/genomic.fna.gz ./genomes
## Rename the genome to the prefix in the config file only
mv GCF_905115235.1_iHerIll2.2.curated.20191125_genomic.fna.gz to GCF_905115235.1.gz


###Run the build command as follows:
java -jar snpEff.jar build -gff3 -v GCF_905115235.1

### Annotate the variants
java -Xmx8g -jar snpEff.jar GCF_905115235.1 var.vcf > annotated.vcf

### Visualization of the output.
The `snpEff`  program performs some statistics and saves them to the file `snpEff_summary.html` on the directory where snpEff is being executed which canbe visualized on your browser.
SnpEff also generates a TXT (tab separated) file having counts of number of variants affecting each transcript and gene. By default, the file name is `snpEff_genes.txt`(Cingolani et al., 2012).

### References
1. De Coster, W., D’Hert, S., Schultz, D. T., Cruts, M., & Van Broeckhoven, C. (2018). NanoPack: visualizing and processing long-read sequencing data. Bioinformatics, 34(15), 2666–2669. https://doi.org/10.1093/BIOINFORMATICS/BTY149


2. Leger, A., & Leonardi, T. (2019). pycoQC, interactive quality control for Oxford Nanopore Sequencing. Journal of Open Source Software, 4(34), 1236. https://doi.org/10.21105/JOSS.01236


3. Payne, A., Holmes, N., Rakyan, V., & Loose, M. (2019). BulkVis: a graphical viewer for Oxford nanopore bulk FAST5 files. Bioinformatics, 35(13), 2193–2198. https://doi.org/10.1093/BIOINFORMATICS/BTY841


4. Wick, R. R., Judd, L. M., Gorrie, C. L., & Holt, K. E. (2017). Completing bacterial genome assemblies with multiplex MinION sequencing. Microbial Genomics, 3(10), e000132. https://doi.org/10.1099/MGEN.0.000132


5. Victoria, D. D. A., Erik, H., Lieven, S., Salvadors, C. G., Cederic, N., Olga, V. P., ... & Henrik, L. (2018). Ten steps to get started in Genome Assembly and Annotation. F1000Research 


6. Cingolani, P., Platts, A., Wang, L. L., Coon, M., Nguyen, T., Wang, L., Land, S. J., Lu, X., & Ruden, D. M. (2012). A program for annotating and predicting the effects of single nucleotide polymorphisms, SnpEff. Fly, 6(2), 80–92. https://doi.org/10.4161/fly.19695


7. Garrison, E., & Marth, G. (2012). Haplotype-based variant detection from short-read sequencing. https://arxiv.org/abs/1207.3907v2
